# Refinitiv Data Library for Python
## Content - Pricing - Streaming

This notebook demonstrates how a Price object to receive streaming real-time data.  

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook - which I have done, and so I don't need to specify a session_type:

In [2]:
# I have defined a default Session type in the Credentials notebook - so no need to provide one here
open_session()

<refinitiv.data.session._platform_session.Definition object at 0x23bd00c7eb0 {session_name='session.platform.1'}>

### Define a callback to receive streaming data

In [3]:
# Function to handle the intial Refresh for each item
def handle_refresh(streaming_prices, instrument_name, fields):
    # One way to access data - get dataframe
    print(f"Refresh : {streaming_prices.get_snapshot()}")
    return

# Function to update dataframe, when we receive updates for individual items
def handle_update(streaming_prices, instrument_name, fields):
    # Alternative way of accesing data - access the updated fields
    print(f"Update : {instrument_name}:{fields}")
    
# Function to extract status code for an item as & when received from server
# Status contains a 'code' and a more detailed 'message'
def handle_status(streaming_prices, instrument_name, status):
    print(f"Status : {instrument_name}:{status['code']}:{status['message']}")

### Create streams for our RICs and open them

In [4]:
# Define our Streaming Price object - including a bad ric for illustrative purposes
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

# Callback for if we wanted to handle invidiual Refresh for each item
streams.on_refresh(handle_refresh)
# Specify callback handler for any updates
streams.on_update(handle_update)
# Specify callback handler for any updates
streams.on_status(handle_status)

# Send the requests to the server and open the streams for all items
streams.open()

Refresh :   Instrument     BID     ASK
0       EUR=  1.1845  1.1849
1       GBP=    <NA>    <NA>
2       JPY=    <NA>    <NA>
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     BID     ASK
0       EUR=  1.1845  1.1849
1       GBP=  1.3922  1.3926
2       JPY=    <NA>    <NA>
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     BID     ASK
0       EUR=  1.1845  1.1849
1       GBP=  1.3922  1.3926
2       JPY=  109.44  109.47
3     BADRIC    <NA>    <NA>
Update : EUR=:{'BID': 1.1846, 'ASK': 1.1847}
Status : BADRIC:NotFound:**The record could not be found


<StreamState.Open: 3>

Update : GBP=:{'BID': 1.3922, 'ASK': 1.3926}
Update : EUR=:{'BID': 1.1845, 'ASK': 1.1849}
Update : GBP=:{'BID': 1.3922, 'ASK': 1.3926}
Update : EUR=:{'BID': 1.1846, 'ASK': 1.1847}
Update : EUR=:{'BID': 1.1844, 'ASK': 1.1848}
Update : JPY=:{'BID': 109.44, 'ASK': 109.46}
Update : EUR=:{'BID': 1.1844, 'ASK': 1.1848}


## Close the stream

In [5]:
streams.close()

Update : JPY=:{'BID': 109.44, 'ASK': 109.47}


<StreamState.Closed: 1>

## Close the session

In [6]:
close_session()